In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score
import kagglehub

In [3]:
data=pd.read_csv('/Indian Sign Language Gesture Landmarks.csv')

In [4]:
data.shape
data.head()
data.describe()

,target,uses_two_hands,left_hand_x_0,left_hand_y_0,left_hand_z_0,left_hand_x_1,left_hand_y_1,left_hand_z_1,left_hand_x_2,left_hand_y_2,...,right_hand_z_17,right_hand_x_18,right_hand_y_18,right_hand_z_18,right_hand_x_19,right_hand_y_19,right_hand_z_19,right_hand_x_20,right_hand_y_20,right_hand_z_20
count,32233.000000,32233.000000,32233.000000,32233.000000,3.223300e+04,32233.000000,32233.000000,32233.000000,32233.000000,32233.000000,...,32232.000000,32232.000000,32232.000000,32232.000000,32232.000000,32232.000000,32232.000000,32232.000000,32232.000000,32232.000000
mean,7.714299,0.753762,0.443477,0.571825,2.885028e-08,0.457219,0.530535,-0.013754,0.465447,0.479331,...,-0.307071,0.164349,0.165741,-0.319677,0.165902,0.174603,-0.313409,0.169287,0.172722,-0.307062
std,4.765505,0.430826,0.205008,0.188382,4.389229e-07,0.150493,0.164875,0.020707,0.115273,0.140078,...,0.397126,0.676797,0.682408,0.390166,0.678095,0.687045,0.393647,0.682171,0.686536,0.397340
min,0.000000,0.000000,-0.004541,-0.016601,-9.644905e-07,0.038794,0.051857,-0.126351,0.092604,0.087936,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,4.000000,1.000000,0.276018,0.447292,-2.847578e-07,0.350360,0.402329,-0.026582,0.386682,0.378121,...,-0.177896,0.170507,0.158336,-0.198187,0.168853,0.201119,-0.183360,0.156759,0.204772,-0.169785
50%,8.000000,1.000000,0.408722,0.566536,5.943169e-08,0.445934,0.521192,-0.009605,0.458340,0.470874,...,-0.086284,0.498405,0.461149,-0.104279,0.498194,0.466575,-0.099714,0.497493,0.459261,-0.094511
75%,12.000000,1.000000,0.613094,0.718964,2.455931e-07,0.569976,0.653940,0.000170,0.543038,0.569453,...,-0.066260,0.599435,0.618798,-0.082462,0.611853,0.636877,-0.076221,0.633070,0.638630,-0.067561
max,16.000000,1.000000,1.050939,1.045371,1.759235e-06,0.964419,0.982267,0.051373,0.897698,0.938779,...,0.016109,0.932940,1.118104,0.033213,0.910637,1.154815,0.050667,0.922977,1.176567,0.058197


In [5]:
scaler=StandardScaler() #is used to normalize the dataset (sets the mean to 0 and unit variance)
X=data.drop('target',axis=1) # it is used to contain every column except the target column
y=data['target'] #it is used to contain the target column
X=scaler.fit_transform(X) # it scandardizes the columns
#Each feature is X'=(X-mean)/std
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42) #splitting the dataset

In [6]:
from xgboost import XGBClassifier # from xgboost lib
from sklearn.model_selection import cross_val_score # is used to evaluate model performance based on different subsets of data
model=XGBClassifier()  #default hyperparameters
cv_scores=cross_val_score(model,X_train,y_train,cv=5,scoring='accuracy') #the data is split into 5 folds (4 of which are for training and one for testing)
# the proccess is repeated for 5 times with accuracy being the metric
print(f'Cross validation scores:{cv_scores}') #cross validation scores
print(f'Mean:{np.mean(cv_scores)}') #mean of the accuracy
print(f'Standard Deviation:{np.std(cv_scores)}') #deviation of accuracy with folds(lower means more consistent)


Cross validation scores:[0.99806126 0.99786698 0.99941827 0.99903044 0.99747915]
Mean:0.9983712212036433
Standard Deviation:0.0007317099329939308


In [7]:
model.fit(X_train,y_train) #trains the model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [8]:
y_pred=model.predict(X_test) #stores the predicted class labels
accuracy=accuracy_score(y_test,y_pred) #stores the accuracy of the model
precision=precision_score(y_test,y_pred,average='weighted') #Precision measures how many of the predicted positive cases were actually correct.
#
recall=recall_score(y_test,y_pred,average='weighted')
print(f'Accuracy:{accuracy}')
print(f'Precision:{precision}')
print(f'Recall:{recall}')

Accuracy:0.9987591127656275
Precision:0.9987610948928382
Recall:0.9987591127656275
